# 2. Getting people data from [The movie DB](https://www.themoviedb.org/). 

We will do this using **The movie DB** API (documentation at [The movie DB API](https://www.themoviedb.org/documentation/api)). Thanks to [galli-leo](https://gist.github.com/galli-leo/6398f9128ffc20af70c6c7eedfeb0a65) for the python script that I slighly adapted for this project to download the entire database of people working on movies industry through **The movie DB** API.  

In [1]:
import requests
import time
import json
import sys
import threading
import queue
from datetime import timedelta

In [1]:
API_KEY = "192f513374c86ad4e8d6f9957ef202e1"
append_to_response = "movie_credits,tv_credits,external_ids,images" 
BASE = "https://api.themoviedb.org/3"
DIR = "./Data/themoviedb/people"

In [3]:
class Worker(threading.Thread):
    def __init__(self, q, *args, **kwargs):
        self.q = q
        super().__init__(*args, **kwargs)

    def run(self):
        while True:
            try:
                work = self.q.get(timeout=3)  # 3s timeout
                do_work(work)
            except queue.Empty:
                return
            # do whatever work you have to do on work
            self.q.task_done()

In [4]:
class TMDBNotfoundException(Exception):
    """docstring for TMDBNotfoundException."""
    def __init__(self):
        super(TMDBNotfoundException, self).__init__()

In [5]:
def get_from_tmdb(path, query, timeout_pause = 5):
    global API_KEY, append_to_response, BASE
    url = BASE + path
    query["api_key"] = API_KEY
    r = requests.get(url, params=query, allow_redirects=True)
    if r.status_code == 404:
        raise TMDBNotfoundException()

    if "X-RateLimit-Remaining" in r.headers:
        rate_limit = r.headers["X-RateLimit-Remaining"]
        if int(rate_limit) <= 2:
            print("Request limit almost reached. Sleeping.")
            time.sleep(timeout_pause)
    else:
        time.sleep(timeout_pause)

    return r.text

In [7]:
def get_people(tmdbid):
    global append_to_response, LATEST_PEOPLE_ID
    return get_from_tmdb("/person/{0}".format(tmdbid), {"append_to_response" : append_to_response, "language" : "en-US", "include_image_language" : "en"})

In [9]:
def get_latest_people_id():
    res = get_from_tmdb("/person/latest", {})
    jres = json.loads(res)
    if not "id" in jres:
        time.sleep(1)
        return get_latest_people_id()
    return jres["id"]

In [10]:
LATEST_PEOPLE_ID = get_latest_people_id()

In [11]:
print(LATEST_PEOPLE_ID)

2120344


In [12]:
def do_update_latest_people_id():
    global LATEST_PEOPLE_ID, q
    while not q.empty():
        lid = get_latest_people_id()
        if lid != LATEST_PEOPLE_ID:
            print("Updating latest id from {0} to {1}".format(LATEST_PEOPLE_ID, lid))
            for newId in range(LATEST_PEOPLE_ID+1, lid+1):
                q.put_nowait(newId)

            LATEST_PEOPLE_ID = lid
        time.sleep(200)


In [13]:
def do_work(tmdbid):
    global LATEST_PEOPLE_ID, start, fromId
    try:
        res = get_people(tmdbid)
        jres = json.loads(res)
        if "id" not in jres:
            time.sleep(1)
            do_work(tmdbid)
            return
        with open(DIR + "/{0}.json".format(tmdbid), "a") as f:
            f.write(res)

        current = time.time()
        difference = float(current - start) / float(tmdbid - fromId + 1)
        eta = (LATEST_PEOPLE_ID - tmdbid) * difference
        print("Downloaded data for person {0} ({1}).".format(jres["name"], tmdbid))
        print("\t  {0}/{1} (Total: {2}/{3}), {4} left.".format(tmdbid - fromId + 1, LATEST_PEOPLE_ID-fromId + 1, tmdbid, LATEST_PEOPLE_ID, str(timedelta(seconds=eta))))
    except TMDBNotfoundException as e:
        print("No person with TMDBID {0} found".format(tmdbid))


In [14]:
def download_all_json():
    global LATEST_PEOPLE_ID, start, fromId, q
    tmdbid = fromId
    start = time.time()
    q = queue.Queue()
    for tmdbid in range(fromId, LATEST_PEOPLE_ID+1):
        q.put_nowait(tmdbid)

    for _ in range(10):
        Worker(q).start()
    try:
        t = threading.Thread(target=do_update_latest_people_id)
        t.start()
        q.join()
    except Exception as e:
        with q.mutex:
            q.queue.clear()
        raise e

In [15]:
fromId = 1
download_all_json()

No person with TMDBID 1099569 found
No person with TMDBID 1099570 found
No person with TMDBID 1099577 found
No person with TMDBID 1099571 found
No person with TMDBID 1099575 found
No person with TMDBID 1099572 found
No person with TMDBID 1099578 found
Updating latest id from 2120344 to 2120345
No person with TMDBID 1099574 foundNo person with TMDBID 1099573 found

No person with TMDBID 1099576 found
No person with TMDBID 1099579 found
No person with TMDBID 1099580 found
No person with TMDBID 1099582 found
No person with TMDBID 1099583 foundNo person with TMDBID 1099581 found

No person with TMDBID 1099588 found
Downloaded data for person Hameed Alam (1099584).
	  16/1020777 (Total: 1099584/2120345), 13 days, 6:50:42.708966 left.
Downloaded data for person Dilip Sen (1099586).
	  18/1020777 (Total: 1099586/2120345), 11 days, 19:38:20.242372 left.
No person with TMDBID 1099589 found
Downloaded data for person Sameer Sen (1099587).
	  19/1020777 (Total: 1099587/2120345), 11 days, 6:53:33.

Downloaded data for person Mercury Sanchez (1099687).
	  119/1020777 (Total: 1099687/2120345), 4 days, 3:30:27.270265 left.
Request limit almost reached. Sleeping.
No person with TMDBID 1099689 found
No person with TMDBID 1099693 found
Downloaded data for person Verónica Álvarez (1099688).
	  120/1020777 (Total: 1099688/2120345), 4 days, 14:33:24.559638 left.
No person with TMDBID 1099697 found
Downloaded data for person Terence M. O'Keefe (1099698).
	  130/1020777 (Total: 1099698/2120345), 4 days, 8:32:20.319702 left.
Downloaded data for person Howard Flaer (1099699).
	  131/1020777 (Total: 1099699/2120345), 4 days, 9:24:10.021299 left.
Downloaded data for person Mark Mercado (1099692).
	  124/1020777 (Total: 1099692/2120345), 4 days, 15:34:23.206928 left.
No person with TMDBID 1099691 found
No person with TMDBID 1099690 found
Downloaded data for person Ira Singerman (1099694).
	  126/1020777 (Total: 1099694/2120345), 4 days, 14:21:54.271664 left.
Downloaded data for person Cameron Cl

Downloaded data for person Erica Brusetti (1099757).
	  189/1020777 (Total: 1099757/2120345), 3 days, 18:38:29.509329 left.
Downloaded data for person Samy Waitzberg (1099756).
	  188/1020777 (Total: 1099756/2120345), 3 days, 19:09:29.762854 left.
Downloaded data for person Robert Matthew Doyle (1099759).
	  191/1020777 (Total: 1099759/2120345), 3 days, 17:54:50.042802 left.
Downloaded data for person Woody Stubblefield (1099760).
	  192/1020777 (Total: 1099760/2120345), 3 days, 17:28:46.632786 left.
Downloaded data for person Carissa Ferreri (1099758).
	  190/1020777 (Total: 1099758/2120345), 3 days, 18:27:12.524238 left.
Downloaded data for person Elijah Rock (1099762).
	  194/1020777 (Total: 1099762/2120345), 3 days, 16:37:24.852431 left.
Downloaded data for person Kwame Siegel (1099761).
	  193/1020777 (Total: 1099761/2120345), 3 days, 17:14:35.386995 left.
Downloaded data for person Brandon J. (1099763).
	  195/1020777 (Total: 1099763/2120345), 3 days, 16:20:35.254697 left.
Reques

	  259/1020777 (Total: 1099827/2120345), 3 days, 16:47:40.768600 left.
Downloaded data for person Mike Seymour (1099829).
	  261/1020777 (Total: 1099829/2120345), 3 days, 16:27:06.276355 left.
Downloaded data for person Joe Vinciguerra (1099830).
	  262/1020777 (Total: 1099830/2120345), 3 days, 16:12:11.856986 left.
Downloaded data for person Ken Christmas (1099831).
	  263/1020777 (Total: 1099831/2120345), 3 days, 15:52:46.770543 left.
Downloaded data for person Frank Vain (1099833).
	  265/1020777 (Total: 1099833/2120345), 3 days, 15:31:13.957840 left.
Downloaded data for person Erika Hampson (1099832).
	  264/1020777 (Total: 1099832/2120345), 3 days, 15:53:23.451119 left.
Downloaded data for person Alexander Grant (1099834).
	  266/1020777 (Total: 1099834/2120345), 3 days, 15:29:09.205924 left.
Downloaded data for person Frank Vain (1099835).
	  267/1020777 (Total: 1099835/2120345), 3 days, 15:18:53.502193 left.
Downloaded data for person Todd Serlin (1099837).
	  269/1020777 (Total

Downloaded data for person Ada Montalvo (1099900).
	  332/1020777 (Total: 1099900/2120345), 3 days, 15:20:10.237563 left.
Downloaded data for person Maya Gnyp (1099904).
	  336/1020777 (Total: 1099904/2120345), 3 days, 14:21:07.503280 left.
Downloaded data for person Maya Gnyp (1099905).
	  337/1020777 (Total: 1099905/2120345), 3 days, 14:09:21.117857 left.
Downloaded data for person Stephen Benwell (1099903).
	  335/1020777 (Total: 1099903/2120345), 3 days, 14:58:16.275797 left.
Downloaded data for person Chris Hocking (1099906).
	  338/1020777 (Total: 1099906/2120345), 3 days, 14:13:54.541420 left.
Downloaded data for person Anita Caplan (1099908).
	  340/1020777 (Total: 1099908/2120345), 3 days, 14:06:15.488556 left.
Downloaded data for person Lora-Mae Adrao (1099907).
	  339/1020777 (Total: 1099907/2120345), 3 days, 14:32:42.796908 left.
Downloaded data for person Ilona Lee (1099911).
	  343/1020777 (Total: 1099911/2120345), 3 days, 13:32:53.079144 left.
Downloaded data for person 

...

No person with TMDBID 1107485 found
Downloaded data for person Liana Mendoza (1107486).
	  7918/1020817 (Total: 1107486/2120385), 3 days, 3:47:46.540844 left.
No person with TMDBID 1107487 found
Downloaded data for person Darla Enlow (1107488).
	  7920/1020817 (Total: 1107488/2120385), 3 days, 3:47:36.756916 left.
Downloaded data for person Vamsikrishna (1107490).
	  7922/1020817 (Total: 1107490/2120385), 3 days, 3:46:56.766895 left.
Downloaded data for person Satya (1107489).
	  7921/1020817 (Total: 1107489/2120385), 3 days, 3:47:42.027633 left.
Downloaded data for person Blake James (1107491).
	  7923/1020817 (Total: 1107491/2120385), 3 days, 3:47:16.114125 left.
Downloaded data for person Suzuki Mint (1107492).
	  7924/1020817 (Total: 1107492/2120385), 3 days, 3:47:40.866175 left.
Downloaded data for person Octavi Pellissa (1107478).
	  7910/1020817 (Total: 1107478/2120385), 3 days, 3:55:54.635878 left.
Downloaded data for person Lola Ferreira (1107481).
	  7913/1020817 (Total: 1107

No person with TMDBID 1107588 found
No person with TMDBID 1107584 found
No person with TMDBID 1107589 found
No person with TMDBID 1107585 found
No person with TMDBID 1107583 found
Downloaded data for person Fred Renata (1107586).
	  8018/1020817 (Total: 1107586/2120385), 3 days, 3:41:20.256562 left.
No person with TMDBID 1107587 found
Downloaded data for person Sivan R (1107591).
	  8023/1020817 (Total: 1107591/2120385), 3 days, 3:39:10.328997 left.
Downloaded data for person Maria Goretti (1107592).
	  8024/1020817 (Total: 1107592/2120385), 3 days, 3:38:41.739073 left.
Downloaded data for person Senthil Kumar (1107590).
	  8022/1020817 (Total: 1107590/2120385), 3 days, 3:39:53.783253 left.
No person with TMDBID 1107594 found
No person with TMDBID 1107593 found
No person with TMDBID 1107597 foundNo person with TMDBID 1107595 found

No person with TMDBID 1107596 found
No person with TMDBID 1107598 found
No person with TMDBID 1107599 found
Downloaded data for person Amy Clover (1107600).

Downloaded data for person Martin Schenk (1107746).
	  8178/1020817 (Total: 1107746/2120385), 3 days, 3:38:12.545769 left.
Downloaded data for person Taekwang Hwang (1107748).
	  8180/1020817 (Total: 1107748/2120385), 3 days, 3:37:12.888546 left.
No person with TMDBID 1107750 found
Downloaded data for person Juris Poškus (1107751).
	  8183/1020817 (Total: 1107751/2120385), 3 days, 3:35:45.937524 left.
Downloaded data for person Lyndsey Love (1107749).
	  8181/1020817 (Total: 1107749/2120385), 3 days, 3:36:57.986832 left.
Downloaded data for person Martin Schenk (1107747).
	  8179/1020817 (Total: 1107747/2120385), 3 days, 3:38:19.695486 left.
No person with TMDBID 1107752 found
Downloaded data for person Frances Lea (1107754).
	  8186/1020817 (Total: 1107754/2120385), 3 days, 3:34:39.516732 left.
No person with TMDBID 1107753 found
Downloaded data for person Judith Johnson (1107755).
	  8187/1020817 (Total: 1107755/2120385), 3 days, 3:34:45.164280 left.
Request limit almost reached. Sle

Downloaded data for person Christy Blair (1107817).
	  8249/1020817 (Total: 1107817/2120385), 3 days, 3:41:22.604690 left.
Downloaded data for person Dido Freire (1107821).
	  8253/1020817 (Total: 1107821/2120385), 3 days, 3:39:26.344289 left.
Downloaded data for person Janine Bazin (1107819).
	  8251/1020817 (Total: 1107819/2120385), 3 days, 3:40:37.093542 left.
Downloaded data for person Guy Polignac (1107822).
	  8254/1020817 (Total: 1107822/2120385), 3 days, 3:39:08.634268 left.
No person with TMDBID 1107823 found
Downloaded data for person Roswitha Soukup (1107824).
	  8256/1020817 (Total: 1107824/2120385), 3 days, 3:39:07.810578 left.
Downloaded data for person Robert Marshall (1107827).
	  8259/1020817 (Total: 1107827/2120385), 3 days, 3:37:45.391090 left.Downloaded data for person David F. Shamoon (1107828).
	  8260/1020817 (Total: 1107828/2120385), 3 days, 3:37:12.946724 left.

Downloaded data for person John Milton Branton (1107826).
	  8258/1020817 (Total: 1107826/2120385), 

KeyboardInterrupt: 